In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
#there are some irregular lines in the data, so setting argument to get a note of which lines and skip them when reading in
accre_gpu = pd.read_csv('../data/accre-gpu-jobs-2022-v2.csv', on_bad_lines='warn')

b'Skipping line 124084: expected 12 fields, saw 13\nSkipping line 124809: expected 12 fields, saw 13\n'


In [3]:
accre_gpu

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66M,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED
1,32880701,glasshouse,brady,0,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32M,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62M,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09M,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336945,46754520_26,harovin,porsha,0,5-00:00:00,00:07:07,1,6,1,turing,0:0,RUNNING
336946,46754542,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336947,46754560,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336948,46754598,glasshouse,treva,145.24M,1-00:00:00,00:01:26,1,5,1,turing,0:0,COMPLETED


In [4]:
accre_gpu.info()
#looking at our dataframe, we will need to adjust some data types to allow calculations

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336950 entries, 0 to 336949
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   JOBID      336950 non-null  object
 1   ACCOUNT    336950 non-null  object
 2   USER       336950 non-null  object
 3   USEDMEM    336950 non-null  object
 4   REQTIME    336950 non-null  object
 5   USEDTIME   336950 non-null  object
 6   NODES      336950 non-null  int64 
 7   CPUS       336950 non-null  int64 
 8   GPUS       336950 non-null  int64 
 9   PARTITION  336950 non-null  object
 10  EXITCODE   336950 non-null  object
 11  STATE      336950 non-null  object
dtypes: int64(3), object(9)
memory usage: 30.8+ MB


In [5]:
#per our stakeholder, no need to include jobs with state of pending or running in our analysis, so dropping those 1258 rows
accre_gpu = accre_gpu.loc[~accre_gpu['STATE'].isin(['PENDING', 'RUNNING'])]

In [6]:
#Confirming that for the USEDMEM column, any values that do not include 'M' are zeroes before stripping the M
accre_gpu.loc[~accre_gpu['USEDMEM'].str.contains('M')]['USEDMEM'].value_counts().sort_values()

0    57489
Name: USEDMEM, dtype: int64

In [7]:
#stripping the 'M' from USEDMEM column and converting it to a numeric data type
accre_gpu['USEDMEM'] = accre_gpu['USEDMEM'].str.strip(to_strip='M').astype(float)

#converting REQTIME and USEDTIME columns to timedelta data types
accre_gpu['REQTIME'] = pd.to_timedelta(accre_gpu['REQTIME'].str.replace('-',' days '))
accre_gpu['USEDTIME'] = pd.to_timedelta(accre_gpu['USEDTIME'].str.replace('-',' days '))

accre_gpu.info()

C:\Users\mews2\AppData\Local\Temp\ipykernel_21880\2731373573.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accre_gpu['USEDMEM'] = accre_gpu['USEDMEM'].str.strip(to_strip='M').astype(float)
C:\Users\mews2\AppData\Local\Temp\ipykernel_21880\2731373573.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accre_gpu['REQTIME'] = pd.to_timedelta(accre_gpu['REQTIME'].str.replace('-',' days '))
C:\Users\mews2\AppData\Local\Temp\ipykernel_21880\2731373573.py:6: SettingWithCopyWarning: 
A value is trying to be s

<class 'pandas.core.frame.DataFrame'>
Int64Index: 335692 entries, 0 to 336948
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype          
---  ------     --------------   -----          
 0   JOBID      335692 non-null  object         
 1   ACCOUNT    335692 non-null  object         
 2   USER       335692 non-null  object         
 3   USEDMEM    335692 non-null  float64        
 4   REQTIME    335692 non-null  timedelta64[ns]
 5   USEDTIME   335692 non-null  timedelta64[ns]
 6   NODES      335692 non-null  int64          
 7   CPUS       335692 non-null  int64          
 8   GPUS       335692 non-null  int64          
 9   PARTITION  335692 non-null  object         
 10  EXITCODE   335692 non-null  object         
 11  STATE      335692 non-null  object         
dtypes: float64(1), int64(3), object(6), timedelta64[ns](2)
memory usage: 33.3+ MB


Doing a little exploration of the values in our columns

In [8]:
accre_gpu['STATE'].value_counts().sort_index(ascending=False)

OUT_OF_MEMORY               3
FAILED                    440
COMPLETED              283333
CANCELLED by 9201       41637
CANCELLED by 913169         3
                        ...  
CANCELLED by 153500         3
CANCELLED by 133418        23
CANCELLED by 111694        11
CANCELLED by 0              1
CANCELLED                 947
Name: STATE, Length: 96, dtype: int64

After dropping jobs marked as pending or running, these are the STATE column values that remain:   
- Out of memory
- Failed
- Completed
- Cancelled
- Cancelled by [numerical code] (92 versions of this)

Exploring the jobs that show some version of a Cancelled state

In [13]:
cancelled = accre_gpu.loc[accre_gpu['STATE'].str.contains('CANCELLED')]
cancelled

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
8,32880758,malanga,arline,0.0,2 days 00:00:00,0 days,0,2,1,pascal,0:0,CANCELLED by 9201
9,32880759,malanga,arline,0.0,2 days 00:00:00,0 days,0,2,1,pascal,0:0,CANCELLED by 9201
10,32880770,malanga,arline,0.0,2 days 00:00:00,0 days,0,2,1,pascal,0:0,CANCELLED by 9201
11,32880771,malanga,arline,0.0,2 days 00:00:00,0 days,0,2,1,pascal,0:0,CANCELLED by 9201
12,32880772,malanga,arline,0.0,2 days 00:00:00,0 days,0,2,1,pascal,0:0,CANCELLED by 9201
...,...,...,...,...,...,...,...,...,...,...,...,...
336807,46745418,malanga,arline,0.0,2 days 00:00:00,0 days,0,2,1,pascal,0:0,CANCELLED by 9201
336830,46745779,malanga,arline,0.0,2 days 00:00:00,0 days,0,2,1,pascal,0:0,CANCELLED by 9201
336832,46745783,malanga,arline,0.0,2 days 00:00:00,0 days,0,2,1,pascal,0:0,CANCELLED by 9201
336880,46752723,glasshouse,ethen,0.0,0 days 04:00:00,0 days,1,6,1,turing,0:0,CANCELLED by 899035


There are 51,916 jobs that have a STATE value of some version of Cancelled. May be worth looking at how much time and memory was spent on these jobs

In [15]:
#taking a quick look just to see how many of the cancelled jobs ran for more than an hour
cancelled.loc[cancelled['USEDTIME'] > '01:00:00'].describe()

,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS
count,295.000000,295,295,295.000000,295.000000,295.000000
mean,7370.163051,3 days 21:08:32.542372881,2 days 10:20:31.593220338,1.471186,5.501695,4.962712
std,11668.831393,1 days 22:49:35.663623084,2 days 01:15:01.972778381,1.016037,5.861714,4.923424
min,0.000000,0 days 01:00:00,0 days 01:03:22,1.000000,1.000000,1.000000
25%,203.060000,2 days 00:00:00,0 days 07:57:40,1.000000,1.000000,1.000000
50%,1236.400000,5 days 00:00:00,2 days 00:00:38,1.000000,4.000000,4.000000
75%,9216.925000,5 days 00:00:00,5 days 00:00:02,1.000000,8.000000,5.000000
max,54338.440000,15 days 00:00:00,9 days 23:59:54,4.000000,33.000000,16.000000


Only 295 Cancelled jobs ran for longer than an hour but the mean and median for these is around 2 days, so, again, may be worth exploring to better understand them and see if such instances can be avoided

In [17]:
cancelled.loc[cancelled['USEDTIME'] > '01:00:00'].groupby(['STATE','EXITCODE']).count()

JOBID  ACCOUNT  USER  USEDMEM  REQTIME  \
STATE               EXITCODE                                           
CANCELLED           0:0          53       53    53       53       53   
                    0:15         96       96    96       96       96   
                    0:6           9        9     9        9        9   
                    0:7           1        1     1        1        1   
                    0:9         114      114   114      114      114   
                    1:0           2        2     2        2        2   
CANCELLED by 505355 0:9           9        9     9        9        9   
CANCELLED by 756088 0:9           1        1     1        1        1   
CANCELLED by 785491 0:9           6        6     6        6        6   
CANCELLED by 859439 0:9           1        1     1        1        1   
CANCELLED by 885613 0:9           3        3     3        3        3   

                              USEDTIME  NODES  CPUS  GPUS  PARTITION  
STATE               EXITCODE                                          
CANCELLED           0:0             53     53    53    53         53  
                    0:15            96     96    96    96         96  
                    0:6              9      9     9     9          9  
                    0:7              1      1     1     1          1  
                    0:9            114    114   114   114        114  
                    1:0              2      2     2     2          2  
CANCELLED by 505355 0:9              9      9     9     9          9  
CANCELLED by 756088 0:9              1      1     1     1          1  
CANCELLED by 785491 0:9              6      6     6     6          6  
CANCELLED by 859439 0:9              1      1     1     1          1  
CANCELLED by 885613 0:9              3      3     3     3          3

Do these exit codes suggest that there tends to be an issue with the environment that is associated with the job being cancelled?

Shifting gears back to the primary dataset

In [20]:
#per our stakeholder, CANCELLED jobs can be excluded from the main analysis, so dropping them from the primary dataframe
accre_gpu = accre_gpu.loc[~accre_gpu['STATE'].str.contains('CANCELLED')]
accre_gpu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283776 entries, 0 to 336948
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype          
---  ------     --------------   -----          
 0   JOBID      283776 non-null  object         
 1   ACCOUNT    283776 non-null  object         
 2   USER       283776 non-null  object         
 3   USEDMEM    283776 non-null  float64        
 4   REQTIME    283776 non-null  timedelta64[ns]
 5   USEDTIME   283776 non-null  timedelta64[ns]
 6   NODES      283776 non-null  int64          
 7   CPUS       283776 non-null  int64          
 8   GPUS       283776 non-null  int64          
 9   PARTITION  283776 non-null  object         
 10  EXITCODE   283776 non-null  object         
 11  STATE      283776 non-null  object         
dtypes: float64(1), int64(3), object(6), timedelta64[ns](2)
memory usage: 28.1+ MB


In [21]:
accre_gpu['EXITCODE'].value_counts()

0:0      283334
1:0         395
2:0          17
127:0         6
7:0           6
13:0          5
126:0         4
16:0          3
0:125         3
33:0          2
9:0           1
Name: EXITCODE, dtype: int64

In [22]:
#Taking a look at the exit codes associated with failed jobs
accre_gpu.loc[accre_gpu['STATE'] == 'FAILED']['EXITCODE'].value_counts()

1:0      395
2:0       17
127:0      6
7:0        6
13:0       5
126:0      4
16:0       3
33:0       2
0:0        1
9:0        1
Name: EXITCODE, dtype: int64

Based on exit codes, the failed jobs seem to fail due to user side issues

In [23]:
#looking at overall distribution of values for our numerical/time columns
accre_gpu.describe()

,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS
count,283776.000000,283776,283776,283776.000000,283776.000000,283776.000000
mean,2261.094059,1 days 00:10:23.951496955,0 days 01:35:33.293763390,1.005490,1.724092,1.089324
std,6639.863578,1 days 11:32:30.215890212,0 days 09:56:52.751470501,0.128331,1.824853,0.859311
min,0.000000,0 days 00:01:00,0 days 00:00:00,0.000000,1.000000,1.000000
25%,7.530000,0 days 10:00:00,0 days 00:00:22,1.000000,1.000000,1.000000
50%,1176.710000,0 days 10:00:00,0 days 00:03:19,1.000000,1.000000,1.000000
75%,2179.940000,0 days 12:00:00,0 days 00:09:08,1.000000,2.000000,1.000000
max,371247.350000,15 days 00:00:00,49 days 02:46:55,15.000000,120.000000,60.000000


In [24]:
#interestingly, values slightly lower if only looking at completed jobs (no failed(440) or out of memory(3))
accre_gpu.loc[accre_gpu['STATE'] == 'COMPLETED'].describe()

,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS
count,283333.000000,283333,283333,283333.000000,283333.000000,283333.000000
mean,2240.911704,1 days 00:05:59.687223161,0 days 01:34:19.081218919,1.004779,1.713161,1.079627
std,6434.562929,1 days 11:29:35.285143482,0 days 09:38:43.694361307,0.115222,1.758140,0.783146
min,0.000000,0 days 00:01:00,0 days 00:00:00,1.000000,1.000000,1.000000
25%,7.530000,0 days 10:00:00,0 days 00:00:22,1.000000,1.000000,1.000000
50%,1176.880000,0 days 10:00:00,0 days 00:03:19,1.000000,1.000000,1.000000
75%,2179.520000,0 days 12:00:00,0 days 00:09:06,1.000000,2.000000,1.000000
max,371247.350000,15 days 00:00:00,8 days 03:15:34,15.000000,120.000000,60.000000


So much skew!

In [25]:
accre_gpu.loc[accre_gpu['USEDTIME'] == accre_gpu['USEDTIME'].max()]

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
317705,45364680,glasshouse,daryl,0.0,0 days 04:00:00,49 days 02:46:55,1,2,1,pascal,0:0,FAILED


In [26]:
accre_gpu.loc[accre_gpu['STATE'] != 'COMPLETED'].describe()

,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS
count,443.000000,443,443,443.000000,443.000000,443.000000
mean,15169.281535,2 days 23:07:21.534988713,0 days 14:46:37.995485327,1.460497,8.715576,7.291196
std,39969.407619,1 days 11:34:52.247900874,2 days 13:01:35.573544238,1.362107,10.367911,6.505382
min,0.000000,0 days 00:10:00,0 days 00:00:00,0.000000,1.000000,1.000000
25%,7.410000,1 days 06:00:00,0 days 00:00:02,1.000000,1.000000,1.000000
50%,210.880000,3 days 08:00:00,0 days 00:07:39,1.000000,4.000000,8.000000
75%,7270.565000,4 days 00:00:00,0 days 08:10:46,2.000000,12.000000,10.000000
max,310280.900000,5 days 00:00:00,49 days 02:46:55,9.000000,70.000000,32.000000


In [47]:
accre_gpu.loc[accre_gpu['USEDTIME'] > '12:00:00']

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
5,32880721,glasshouse,brady,403.40,3 days 12:00:00,0 days 18:49:36,1,1,1,pascal,0:0,COMPLETED
6,32880722,glasshouse,brady,0.00,3 days 12:00:00,3 days 12:00:07,1,1,1,pascal,0:0,COMPLETED
327,32884086,glasshouse,miya,374.89,5 days 00:00:00,1 days 08:38:34,1,1,1,pascal,0:0,COMPLETED
540,32885557,malanga,alwilda,7572.38,2 days 00:00:00,1 days 05:49:07,1,2,1,pascal,0:0,COMPLETED
784,32888937,virginia,almond,0.00,2 days 12:00:00,2 days 12:00:10,1,18,3,turing,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
335986,46586557_36,harovin,porsha,39073.00,5 days 00:00:00,1 days 22:08:42,1,6,1,turing,0:0,COMPLETED
336004,46594210,pecans,ramona,2966.58,2 days 00:00:00,1 days 21:11:57,1,2,1,pascal,0:0,COMPLETED
336005,46594666,glasshouse,santiago,82.06,2 days 00:00:00,2 days 00:00:29,1,24,4,turing,0:0,COMPLETED
336035,46606714,glasshouse,mel,11823.67,5 days 00:00:00,0 days 12:00:48,3,12,16,pascal,0:0,COMPLETED


Approximately 2.8% of jobs that show a status of completed, failed, or out of memory run for longer than 12 hours  
Approximately 1.6% run longer than 24 hours

In [46]:
accre_gpu.loc[accre_gpu['PARTITION'] == 'maxwell'].describe()

,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS
count,127160.000000,127160,127160,127160.000000,127160.000000,127160.000000
mean,1052.447286,0 days 15:06:01.807172066,0 days 00:29:55.219479396,1.003154,1.146005,1.052359
std,2331.444456,0 days 22:31:52.030182916,0 days 05:24:04.844920416,0.092062,1.344548,0.632417
min,0.000000,0 days 00:01:00,0 days 00:00:00,0.000000,1.000000,1.000000
25%,5.540000,0 days 10:00:00,0 days 00:00:10,1.000000,1.000000,1.000000
50%,1113.155000,0 days 10:00:00,0 days 00:03:01,1.000000,1.000000,1.000000
75%,1315.842500,0 days 10:00:00,0 days 00:03:45,1.000000,1.000000,1.000000
max,119399.180000,15 days 00:00:00,5 days 00:00:32,10.000000,80.000000,40.000000


In [44]:
accre_gpu.loc[accre_gpu['PARTITION'] == 'pascal'].describe()

,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS
count,81544.000000,81544,81544,81544.000000,81544.000000,81544.000000
mean,2692.467733,1 days 23:55:11.727901501,0 days 02:52:43.346720788,1.012214,1.506892,1.169050
std,6611.661783,1 days 20:18:05.288080371,0 days 13:10:13.568448855,0.194407,1.605479,1.260511
min,0.000000,0 days 00:01:00,0 days 00:00:00,0.000000,1.000000,1.000000
25%,164.490000,0 days 10:00:00,0 days 00:02:36,1.000000,1.000000,1.000000
50%,1363.295000,1 days 00:00:00,0 days 00:08:36,1.000000,1.000000,1.000000
75%,3255.512500,3 days 00:00:00,0 days 00:30:54,1.000000,2.000000,1.000000
max,248467.890000,12 days 00:00:00,49 days 02:46:55,15.000000,120.000000,60.000000


In [48]:
accre_gpu.loc[accre_gpu['PARTITION'] == 'turing'].describe()

,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS
count,75072.000000,75072,75072,75072.000000,75072.000000,75072.000000
mean,3839.786365,0 days 13:44:50.690537084,0 days 02:02:54.551324062,1.002145,2.939205,1.065337
std,10237.039246,1 days 06:22:33.444369635,0 days 11:29:55.807817987,0.082314,2.152269,0.614944
min,0.000000,0 days 00:01:00,0 days 00:00:00,1.000000,1.000000,1.000000
25%,7.410000,0 days 00:30:00,0 days 00:00:14,1.000000,2.000000,1.000000
50%,1253.960000,0 days 01:00:00,0 days 00:03:02,1.000000,2.000000,1.000000
75%,2420.410000,0 days 12:00:00,0 days 00:15:32,1.000000,4.000000,1.000000
max,371247.350000,11 days 00:00:00,5 days 00:04:26,10.000000,80.000000,40.000000
